In [3]:
import json
import tqdm
from flair.data import Sentence

/home/jovyan/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
2023-10-29 09:40:20.946787: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 09:40:21.194324: I tensorflow/core/util/port.cc:104] oneDNN custom operations are 

In [1]:
def read_data(input_file_path):
    data = []

    with open(input_file_path) as f:
        for line in f:
            data.append(json.loads(line))

    return data

In [2]:
def write_data(output_file_path, data):
    # Open a new JSON file for writing
    with open(output_file_path, "w") as output_file:
        for data_line in data:
            output_file.write(json.dumps(data_line) + "\n")

    print(f"Data with 'Test' property added has been saved to {output_file_path}")

In [394]:
data = read_data("./squad_data_train_pos_ner.json")

In [395]:
def is_subustring_in_string(string, substring, symbol):
    # print(string, symbol, symbol in string and substring in string)
    return symbol in string and substring in string


def is_one_text_in_context(text, context):
    if context.count(text+' ') +  context.count(text+'.') > 1:
        return False
    # print(text, context.count(text+' '), context.count(text+'-'), context.count(text+'.'))
    return context.count(text+' ') == 1 or context.count(text+'-') == 1 or context.count(text+'.') == 1

In [396]:
def find_start_position(text, pos_context, answer, context):
    # print(text)
    sentence = Sentence(text)
    tokens = []
    spaces_no = context[:answer].count(" ")
    # pos_context = pos_context.replace('\\','')

    for word in sentence:
        tokens.append(word.text)
    # print(tokens)

    start_position = 0
    token_number = 0
    match = False
    no_match = -1
    unicode = False
    # print(context.count(text))
    one_apperance = is_one_text_in_context(text, context)
    # print(one_apperance)
    for start_position in range(len(pos_context) - len(tokens) + 1):
        k_words_context_ = [bytes(list(line.values())[0][0], "utf-8").decode("unicode_escape") for line in
                      pos_context[start_position:start_position + len(tokens)]]
        k_words_context = [list(line.values())[0][0].replace('\\','') for line in
                      pos_context[start_position:start_position + len(tokens)]]
        next_word = [list(line.values())[0][0] for line in
                      pos_context[start_position + len(tokens):start_position + len(tokens) + 1]]
        # print(k_words_context, tokens, token_number + spaces_no)
        k_words_context_string = " ".join(k_words_context)
        if context.count(text) == 1 and k_words_context == tokens:
            match = True
            break

        if one_apperance and (k_words_context == tokens or is_subustring_in_string(k_words_context_string, text, '-') or is_subustring_in_string(k_words_context_string, text, '.')):
            # print(text, is_one_text_in_context(text, context), " ".join(tokens),  " ".join(k_words_context), token_number + spaces_no)
            match = True
            break

        if text in k_words_context_string and (token_number + spaces_no == answer or token_number + spaces_no + 1 == answer):
            match = True
            break

        # if (token_number + spaces_no == answer):
        #     no_match = start_position
        #     print('HI 1')
        #     break
        elif next_word == ' ' and token_number + spaces_no + 1 == answer:
            no_match = start_position
            break
        if "'" in k_words_context_:
            token_number += len(k_words_context_[0]) + 1
        else:
            token_number += len(k_words_context_[0])
    if match:
        return start_position
    return no_match

In [397]:
elem = 292
data[elem]["answers"]

{'text': ['Dreamgirls'], 'answer_start': [260]}

In [398]:
data[elem]["context"]

'Following the disbandment of Destiny\'s Child in June 2005, she released her second solo album, B\'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploratio

In [399]:
index = find_start_position(data[elem]["answers"]["text"][0],data[elem]["POS_context"], data[elem]["answers"]["answer_start"][0], data[elem]["context"])
index

53

In [400]:
data[elem]['POS_context'][index][str(index)]

['Dreamgirls', 'NNPS']

In [401]:
special_characters = ["$", ",", "''", "-LRB-", "-RRB-", ".", ":", "``"]


def get_proc_pos_between_item(pos_context, context, answers):
    positions = {}
    copy_answer = []
    i = 0
    # answers["answer_start"].sort()
    # answers["answer_start"].sort()
    
    for answer in answers["answer_start"]:
        answer_position = find_start_position(answers["text"][i], pos_context, answer, context)
        if answer_position != -1:
            positions[answer_position] = answers["answer_start"][i]
            copy_answer.append(answers["answer_start"][i])
        i += 1
    pos = ""
    offset = 0
    offsets = {}
    # print(positions)
    for index in range(len(pos_context)):
        pos_text = pos_context[index][str(index)]

        if pos_text[1] in special_characters:
            pos_text[1] = "sym"
        if index in positions.keys():
            offsets[positions[index]] = offset
        pos += f"{pos_text[0]} [{pos_text[1].lower()}] "
        offset += len(f" [{pos_text[1]}] ")

    answer_starts = []
    # print(copy_answer)
    for index in range(len(copy_answer)):
        answer = copy_answer[index]
        if answer == 0:
            answer_starts.append(answer)
            continue
        spaces_no = context[:answer].count(" ") + 1
        if answer not in offsets.keys():
            continue
        else:
            answer += offsets[answer] - spaces_no
        if pos[answer : answer + 1] == " ":
            answer += 1
        # if len(answers["text"]) > index:
        #     if len(answers["text"][index]) > 1:
        #         if pos[answer][0] == answers["text"][index][1]:
        #             answer -= 1
        answer_starts.append(answer)

    return pos, answer_starts


In [402]:
example_id = 84

pos, answer_starts = get_proc_pos_between_item(data[example_id]['POS_context'], data[example_id]['context'],
                          data[example_id]['answers'])
pos, answer_starts

('The [dt] university [nn] is [vbz] affiliated [vbn] with [in] the [dt] Congregation [nnp] of [in] Holy [nnp] Cross [nnp] ( [sym] Latin [nnp] : [sym] Congregatio [nnp] a [dt] Sancta [nnp] Cruce [nnp] , [sym] abbreviated [vbn] postnominals [nns] : [sym] " [sym] CSC [nnp] ") [sym] . [sym] While [in] religious [jj] affiliation [nn] is [vbz] not [rb] a [dt] criterion [nn] for [in] admission [nn] , [sym] more [jjr] than [in] 93 [cd] % [nn] of [in] students [nns] identify [vbp] as [in] Christian [nnp] , [sym] with [in] over [in] 80 [cd] % [nn] of [in] the [dt] total [nn] being [vbg] Catholic [jj] . [sym] Collectively [rb] , [sym] Catholic [nnp] Mass [nnp] is [vbz] celebrated [vbn] over [in] 100 [cd] times [nns] per [in] week [nn] on [in] campus [nn] , [sym] and [cc] a [dt] large [jj] campus [nn] ministry [nn] program [nn] provides [vbz] for [in] the [dt] faith [nn] needs [nns] of [in] the [dt] community [nn] . [sym] There [ex] are [vbp] multitudes [nns] of [in] religious [jj] statues [nns] a

In [403]:
pos[554:]

'[dt] total [nn] being [vbg] Catholic [jj] . [sym] Collectively [rb] , [sym] Catholic [nnp] Mass [nnp] is [vbz] celebrated [vbn] over [in] 100 [cd] times [nns] per [in] week [nn] on [in] campus [nn] , [sym] and [cc] a [dt] large [jj] campus [nn] ministry [nn] program [nn] provides [vbz] for [in] the [dt] faith [nn] needs [nns] of [in] the [dt] community [nn] . [sym] There [ex] are [vbp] multitudes [nns] of [in] religious [jj] statues [nns] and [cc] artwork [nn] around [in] campus [nn] , [sym] most [rbs] prominent [jj] of [in] which [wdt] are [vbp] the [dt] statue [nn] of [in] Mary [nnp] on [in] the [dt] Main [nnp] Building [nnp] , [sym] the [dt] Notre [nnp] Dame [nnp] Grotto [nnp] , [sym] and [cc] the [dt] Word [nnp] of [in] Life [nnp] mural [nn] on [in] Hesburgh [nnp] Library [nnp] depicting [vbg] Christ [nnp] as [in] a [dt] teacher [nn] . [sym] Additionally [rb] , [sym] every [dt] classroom [nn] displays [vbz] a [dt] crucifix [nn] . [sym] There [ex] are [vbp] many [jj] religious [jj]

In [404]:
for item in tqdm.tqdm(data):
    context = item['context']
    question = item['question']
    item['context'], item['answers']['answer_start'] = get_proc_pos_between_item(item['POS_context'], context, item['answers'])

    item.pop('POS_context', None)
    item.pop('POS_question', None)
    item.pop('NER_question', None)
    item.pop('NER_context', None)

100%|██████████| 87599/87599 [01:00<00:00, 1458.73it/s]


In [405]:
incorrect_answers_no = 0

for index, line in tqdm.tqdm(enumerate(data)):
    try:
        if len(line["answers"]["answer_start"]) == 0:
            continue
        answer_start = line["answers"]["answer_start"][0]
        answer = line["answers"]["text"][0]

        if answer_start == -1:
            continue

        if line["context"][answer_start:][0] != answer[0]:
            # line["answers"]["answer_start"][0] -= 1
            incorrect_answers_no += 1
            #print(index, line["context"][answer_start:], 4*'-', answer, answer_start)

    except Exception as e: 
        print(e)

87599it [00:00, 183267.93it/s]


In [406]:
incorrect_answers_no = 0

for i, line in tqdm.tqdm(enumerate(data)):
    try:
        if(len(line["answers"]["answer_start"]) == 0):
            line["answers"]["answer_start"].append(0)
        answer_start = line["answers"]["answer_start"][0]
        answer =  line["answers"]["text"][0]
        answer_length = len(answer)

        if line["context"][answer_start:][0] != answer[0]:
            incorrect_answers_no += 1

    except Exception as e: 
        # print(e)
        incorrect_answers_no += 1
        # print(i, answer_start, line["context"][answer_start:], 4*'-', answer)

87599it [00:00, 270431.26it/s]


In [407]:
data[292]

{'id': '56bf6e823aeaaa14008c9627',
 'title': 'Beyoncé',
 'context': 'Following [vbg] the [dt] disbandment [nn] of [in] Destiny [nnp] \'s [pos] Child [nnp] in [in] June [nnp] 2005 [cd] , [sym] she [prp] released [vbd] her [prp$] second [jj] solo [jj] album [nn] , [sym] B\'Day [nnp] ( [sym] 2006 [cd] ) [sym] , [sym] which [wdt] contained [vbd] hits [nns] " [sym] Déjà [nnp] Vu [nnp] " [sym] , [sym] " [sym] Irreplaceable [nnp] " [sym] , [sym] and [cc] " [sym] Beautiful [nnp] Liar [nnp] " [sym] . [sym] Beyoncé [nnp] also [rb] ventured [vbd] into [in] acting [nn] , [sym] with [in] a [dt] Golden [nnp] Globe-nominated [nnp] performance [nn] in [in] Dreamgirls [nnps] ( [sym] 2006 [cd] ) [sym] , [sym] and [cc] starring [vbg] roles [nns] in [in] The [dt] Pink [nnp] Panther [nnp] ( [sym] 2006 [cd] ) [sym] and [cc] Obsessed [nnp] ( [sym] 2009 [cd] ) [sym] . [sym] Her [prp$] marriage [nn] to [in] rapper [nn] Jay [nnp] Z [nnp] and [cc] portrayal [nn] of [in] Etta [nnp] James [nnp] in [in] Cadillac [n

In [408]:
incorrect_answers_no

2390

In [409]:
write_data("./squad_data_validation_pos_between.json", data)

Data with 'Test' property added has been saved to ./squad_data_validation_pos_between.json


In [410]:
proc_data = read_data('./squad_data_validation_pos_between.json')

In [411]:
def rreplace(string, substring, replacement):
    k = string.rfind(substring)
    return string[:k] + replacement + string[k+len(substring):]

In [412]:
symbols = ["'", '.', ',', ')']


def get_updated_answer(context, answers):
    new_answers = []
    if len(answers['answer_start']) == 0:
        return

    for index, answer in enumerate(answers['text']):
        new_answer = ''
        if len(answers['answer_start']) < index + 1:
            return new_answers
        answer_start = answers['answer_start'][index]
        if answer_start != -1:
            tokenized_context = Sentence(context[answer_start:])
            no_of_words = len(Sentence(answer))
            max_length = no_of_words * 3 + no_of_words
            if max_length > len(tokenized_context):
                max_length = len(tokenized_context)
            minus_words = 0
            for token_index in range(max_length):
                # if token_index == max_length - minus_words - 1:
                #     break
                if (tokenized_context[token_index].text == '[' and tokenized_context[token_index+1].text != '[') or (len(tokenized_context) > token_index+1 and tokenized_context[token_index].text!=']' and tokenized_context[token_index+1].text==']') or token_index == no_of_words * 4 - 1:
                    new_answer += tokenized_context[token_index].text
                elif tokenized_context[token_index].text in symbols and tokenized_context[token_index+1].text != '[':
                    new_answer += tokenized_context[token_index].text
                elif (tokenized_context[token_index].text.replace('.', '').isalpha() or tokenized_context[token_index].text[-1] in '."') and tokenized_context[token_index+1].text in '.$]':
                    new_answer += tokenized_context[token_index].text
                    # minus_words += 1
                elif len(tokenized_context) > token_index+1 and tokenized_context[token_index].text.replace('.', '').isnumeric() and tokenized_context[token_index+1].text in '.$]':
                    new_answer += tokenized_context[token_index].text
                elif tokenized_context[token_index].text.replace('-', '').isalpha() and tokenized_context[token_index+1].text in '.$]':
                    new_answer += tokenized_context[token_index].text
                elif len(tokenized_context) > token_index+1 and tokenized_context[token_index+1].text[0].isnumeric() and tokenized_context[token_index].text in '-':
                    new_answer += tokenized_context[token_index].text
                else:
                    new_answer += tokenized_context[token_index].text + ' '
        new_answer = new_answer.rstrip()
        if answer_start != -1:
            tokens_new_answer = Sentence(new_answer)
            # print(tokens_new_answer[-2])
            if tokens_new_answer[-2].text == '[':
                # print(tokens_new_answer[-2])
                new_answer += ']'
            if tokens_new_answer[-1].text == '[':
                #print(tokens_new_answer[-1], tokens_new_answer[-2].text)
                new_answer = rreplace(new_answer, tokens_new_answer[-1].text, '')
                new_answer = rreplace(new_answer, tokens_new_answer[-2].text, '')
                new_answer = new_answer.rstrip()
        new_answers.append(new_answer)
    return new_answers

In [413]:
example_id = 4892

In [414]:
get_updated_answer(proc_data[example_id]['context'], proc_data[example_id]['answers'])

['a [dt] U.S. [nnp] inventor [nn] , [sym] engineer [nn] and [cc] solar [jj] energy [nn] pioneer [nn]']

In [ ]:
for item in tqdm.tqdm(proc_data):
    item['answers']['text'] = get_updated_answer(item['context'], item['answers'])

 93%|█████████▎| 81573/87599 [03:34<00:16, 362.93it/s]

In [391]:
incorrect_answers_no = 0

for index, line in tqdm.tqdm(enumerate(proc_data)):
    try:
        if len(line["answers"]["text"]) == 0:
            line["answers"]["text"] = ['']
        if len(line["answers"]["answer_start"]) == 0:
            line["answers"]["text"] = [0]
        answer_start = line["answers"]["answer_start"][0]
        answer = line["answers"]["text"][0]
        answer_length = len(answer)

        if answer_start == -1:
            continue

        if line["context"][answer_start: answer_start + answer_length] != answer:
            incorrect_answers_no += 1
            print(index, line["context"][answer_start: answer_start + answer_length],
                  "-" * 4, answer)

        # if answer[-1] != ']':
        #     print(index, line["context"][answer_start: answer_start + answer_length], "-" * 4, answer)
        #     incorrect_answers_no += 1

    except Exception as e: 
        print(e)

10570it [00:00, 284819.05it/s]

717 :28 [cd]  ---- : 28 [cd]
3615 oxygen-18. [nn]  ---- oxygen-18 . [nn]
4006 -Gemini [jj]  ---- - Gemini [jj]


In [392]:
incorrect_answers_no / len(proc_data) * 100

0.028382213812677387

In [393]:
write_data("./squad_data_train_pos_ner_answer_updated.json", proc_data)

Data with 'Test' property added has been saved to ./squad_data_validation_pos_ner_answer_updated.json
